In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hg5c_6w5
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hg5c_6w5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-r9u8f0it/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpf0n1ailz".


In [14]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

char* CpuCrypt(char* rawPassword) {
    char* newPassword = (char *) malloc(sizeof(char) * 11);

    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    for(int i = 0; i < 10; i++){
        if(i < 6){ //checking letter limits
            if(newPassword[i] > 122){
                newPassword[i] = (newPassword[i] - 122) + 97;
            } else if(newPassword[i] < 97){
                newPassword[i] = (97 - newPassword[i]) + 97;
            }
        } else { //checking numbers
            if(newPassword[i] > 57){
                newPassword[i] = (newPassword[i] - 57) + 48;
            } else if(newPassword[i] < 48){
                newPassword[i] = (48 - newPassword[i]) + 48;
            }
        }
    }
    return newPassword;
}

int main(int argc, char** argv) {
    const char* encryptedPassword = "ccbddb7362";
    if (argc > 1) {
        encryptedPassword = argv[1];
    }
    printf("Encrypted Password: '%s'\n", encryptedPassword);

    char cpuAlphabet[26] = { 'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z' };
    char cpuNumbers[10] = { '0', '1', '2', '3', '4', '5', '6' ,'7', '8', '9' };

    char cpuOutputPass[11]; // Allocate space for the output password

    // Brute force decryption
    for (int i = 0; i < 26; i++) {
        for (int j = 0; j < 26; j++) {
            for (int k = 0; k < 10; k++) {
                for (int l = 0; l < 10; l++) {
                    char potentialPassword[5];
                    potentialPassword[0] = cpuAlphabet[i];
                    potentialPassword[1] = cpuAlphabet[j];
                    potentialPassword[2] = cpuNumbers[k];
                    potentialPassword[3] = cpuNumbers[l];
                    potentialPassword[4] = '\0';

                    char* encryptedPotential = CpuCrypt(potentialPassword);

                    if (strcmp(encryptedPotential, encryptedPassword) == 0) {
                        printf("Found password: '%s' for encrypted password: '%s'\n", potentialPassword, encryptedPassword);
                        strcpy(cpuOutputPass, potentialPassword);
                        break;
                    }

                    free(encryptedPotential);
                }
            }
        }
    }

    printf("Results:\n");
    if (cpuOutputPass[0] != 0) {
        printf("Decrypted Password: '%s'\n", cpuOutputPass);
    } else {
        printf("Password decryption failed.\n");
    }

    return 0;
}


Encrypted Password: 'ccbddb7362'
Found password: 'aa52' for encrypted password: 'ccbddb7362'
Results:
Decrypted Password: 'aa52'

